In [ ]:
!pip install flask flask-ngrok
!pip install google-cloud-aiplatform
!pip install google-generativeai
import base64
import json
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok


from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import google.generativeai as genai
model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = "W3sicGFydHMiOiBbeyJ0ZXh0IjogIldoYXQncyBpbiB0aGlzIHBpY3R1cmU/In0sIHsiZmlsZV9kYXRhIjogeyJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2dlbmVyYXRpdmVhaS1kb3dubG9hZHMvaW1hZ2VzL3Njb25lcy5qcGciLCAibWltZV90eXBlIjogImltYWdlL2pwZWcifX1dfV0="  # @param {isTemplate: true}
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False
genai.configure(api_key="Enter your API KEY")

In [ ]:
!pip install scikit-learn==1.3.1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import io
import base64
train_df = pd.read_csv('/content/staff_present_2015_2021_realistic.csv')
event_df = pd.read_csv('/content/public_holidays.csv')
train_df = train_df.dropna(subset=['Present'])
# Step 3: Preprocessing Data
# Convert 'date' columns to datetime
def create_date_dataframe(start_date, end_date):
    start_date = pd.to_datetime(start_date,format='%d/%m/%Y')  # Convert start_date to datetime
    end_date = pd.to_datetime(end_date,format='%d/%m/%Y')
    dates = pd.date_range(start=start_date, end=end_date, freq='D')  # Generate date range
    df = pd.DataFrame({'Date': dates})  # Create DataFrame
    return df


def pre_process_data(date_df,event_df=event_df):
 date_df['Date'] = pd.to_datetime(date_df['Date'], format='%Y-%m-%d')
 event_df['Date'] = pd.to_datetime(event_df['Date'], format='%Y-%m-%d')
# Extract more date-related features
 date_df['day_of_week'] = date_df['Date'].dt.dayofweek
 date_df['month'] = date_df['Date'].dt.month
 date_df['day'] = date_df['Date'].dt.day
 date_df['weekofyear'] = date_df['Date'].dt.isocalendar().week
 date_df['quarter'] = date_df['Date'].dt.quarter
 date_df['is_weekend'] = date_df['day_of_week'].isin([5, 6]).astype(int)
 date_df['is_month_start'] = date_df['Date'].dt.is_month_start.astype(int)
 date_df['is_month_end'] = date_df['Date'].dt.is_month_end.astype(int)
 date_df['is_holiday'] = (date_df['Date'].isin(event_df['Date'])).astype(int)
 date_df['is_next_holiday'] = (date_df['Date'].isin(event_df['Date'] + pd.Timedelta(days=1))).astype(int)
 date_df['is_prev_holiday'] = (date_df['Date'].isin(event_df['Date'] - pd.Timedelta(days=1))).astype(int)
 date_df['is_sunday'] = (date_df['day_of_week'] == 6).astype(int)
 date_df["Total"] = date_df.apply(lambda row: 0 if row["is_holiday"] == 1 or row["day_of_week"] == 6 else 100, axis=1)
pre_process_data(train_df)
# Step 5: Feature Engineering
# Select features and target variable
features = ['day_of_week', 'month', 'day', 'weekofyear', 'quarter', 'is_weekend', 'is_month_start', 'is_month_end', 'is_holiday','is_next_holiday','is_prev_holiday',"is_sunday"]
X_train = train_df[features]
y_train = train_df['Present']
#MODEL
xgb_model = XGBRegressor(random_state=42,colsample_bytree= 1.0, learning_rate= 0.8, max_depth= 3, n_estimators=80, subsample= 0.4 )
xgb_model.fit(X_train, y_train)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.8, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=80, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
def timeseries(start_date,end_date):
    date_df = create_date_dataframe(start_date, end_date)
    pre_process_data(date_df)
    forecast=xgb_model.predict(date_df[features])
    forecast_round=np.floor(forecast)
    submission = pd.DataFrame({
    'Date': date_df['Date'],
    'Present': forecast_round.astype(int)})

    # Convert 'Date' column to datetime for plotting
    submission['Date'] = pd.to_datetime(submission['Date'], format='%d-%m-%Y')
    submission = submission.set_index('Date')
    submission['Present'].plot()
    plt.xlabel("Date")
    plt.ylabel("Present")
    plt.title("Employee Presence")
    # Save the plot to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # Encode the image data to base64
    image_base64 = base64.b64encode(buf.read()).decode('utf-8')
    plt.clf()
    plt.close('all')
    return image_base64


def date_estimation(start_date,end_date):
    date_df = create_date_dataframe(start_date, end_date)
    pre_process_data(date_df)
    end_date_extra = pd.to_datetime(end_date,format="%d/%m/%Y") + pd.DateOffset(months=3)
    df = create_date_dataframe(start_date, end_date_extra)
    pre_process_data(df)
    forecast=xgb_model.predict(df[features])
    forecast_round=np.floor(forecast)
    submission = pd.DataFrame({
    'Date': df['Date'],
    'Present': forecast_round.astype(int)
})
    cumulative_sum = 0
    target_sum = date_df['Total'].sum()
    target_date = None
    for index, row in submission.iterrows():
         cumulative_sum += row['Present']
         if cumulative_sum >= target_sum:
             target_date = row['Date']
             break
    estdate=str(target_date)
    output=estdate
    return output
def get_gemini_prompt(variable):
    return (
        f"You must analyze the input text and extract data to determine whether the request is for a time series prediction "
        f"or a project completion date estimation. Based on your analysis, return a structured JSON response in the following format:\n\n"
        f"1. For time series prediction:\n"
        f"{{\n"
        f"  \"type\": 1,\n"
        f"  \"start_date\": \"dd/mm/yyyy\",\n"
        f"  \"end_date\": \"dd/mm/yyyy\",\n"
        f"  \"response\": \"[Realistic response related to the time series plot]\"\n"
        f"}}\n\n"
        f"2. For project completion date estimation:\n"
        f"{{\n"
        f"  \"type\": 2,\n"
        f"  \"start_date\": \"dd/mm/yyyy\",\n"
        f"  \"end_date\": \"dd/mm/yyyy\",\n"
        f"  \"response\": \"[Realistic response related to project completion date, with placeholder for estimated date: '{{estimated_date}}']\"\n"
        f"}}\n\n"
        f"Instructions:\n"
        f"- Analyze the text to determine the type of request (time series prediction or project estimation).\n"
        f"- For time series prediction:\n"
        f"  - Extract the start and end dates from the text. If the request specifies a specific month, week, or year (like 'this month,' 'this year,' or 'this week'), use the current date (e.g., current year 2025) to fill in the date range.\n"
        f"  - Provide a realistic response with a plot description, for example: 'Here is the plot for [requested data] in the tone and style of the input.'\n"
        f"- For project estimation:\n"
        f"  - Extract the start date and the estimated time to complete the project (in months, days, or other units). Convert the time to months or days as needed and calculate the end date accordingly.\n"
        f"  - Provide a realistic response in the tone of the query, such as: 'Based on your input, the project will be completed on {{estimated_date}}.'\n"
        f"- Ensure the dates are in \"dd/mm/yyyy\" format.\n"
        f"- The response must include a field called 'response' that provides a realistic, natural language answer to the query, in the same tone and style as the input.\n"
        f"- Do not include any additional text, symbols, or explanations in your response. Only return the JSON object.Also do not json in front just the object i.e starting with curly braces\n\n"
        f"- If theres no enough details or cannot perform function or prompt is not correct give a valid response for it based on the input in the response field and turn all other three fields to 0\n\n"
        f"Input example: \"Predict the time series for employee leave data for March 2025.\"\n"
        f"Output example: \n"
        f"{{\n"
        f"  \"type\": 1,\n"
        f"  \"start_date\": \"01/03/2025\",\n"
        f"  \"end_date\": \"31/03/2025\",\n"
        f"  \"response\": \"Here is the plot for employee leave data for March 2025.\"\n"
        f"}}\n\n"
        f"Input example: \"The project starts on 15/01/2025 and is estimated to take 3 months.\"\n"
        f"Output example:\n"
        f"{{\n"
        f"  \"type\": 2,\n"
        f"  \"start_date\": \"15/01/2025\",\n"
        f"  \"end_date\": \"15/04/2025\",\n"
        f"  \"response\": \"Based on your input, the project will be completed on {{estimated_date}}.\"\n"
        f"}}\n\n"
        f"Here’s the text: {variable}"
    )



In [ ]:
import subprocess
!pip install Flask-Cors # Install the flask_cors package
!pip install pyngrok # Install the pyngrok package
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import json
import os
import subprocess
!pip install Flask-Cors # Install the flask_cors package
!pip install pyngrok # Install the pyngrok package
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import json
import os


app = Flask(__name__)
@app.route('/process', methods=['POST'])
def process_prompt():
    data = request.json
    user_prompt = data.get('prompt')

    if not user_prompt:
        return jsonify({"error": "Missing 'prompt'"}), 400
    else:
        gemini = genai.GenerativeModel(model_name=model)
        prompt=get_gemini_prompt(user_prompt)
        response = gemini.generate_content({prompt},generation_config=generation_config,safety_settings=safety_settings,stream=False)
        rt=response.text
        data=json.loads(rt)
        #print(data)
        s=data['start_date']
        e=data['end_date']
        resp_out=data['response']
        if data['type']==1:
            #print("inside type 1")
            start = pd.to_datetime(s, format='%d/%m/%Y')
            end = pd.to_datetime(e, format='%d/%m/%Y')
            output=timeseries(start,end)
            #print(resp_out)
            return jsonify({"base64":output,"result":resp_out})
        elif data['type']==2:
            print("inside type 2")
            start = pd.to_datetime(s, format='%d/%m/%Y')
            end = pd.to_datetime(e, format='%d/%m/%Y')
            output=date_estimation(start,end)
            resp_out=str(resp_out)
            resp_outn =resp_out.format(estimated_date=output)
            #print(resp_out)
            #print(output)
            return jsonify({"result": resp_outn})
        else:
             output="Sorry i cant do this"
             print (data['type'])
             #print(data['type'].type())
             return jsonify({"result": resp_out})



# Setup ngrok
def setup_ngrok():
    # Set your authtoken (you only need to do this once)
    # Get your authtoken from https://dashboard.ngrok.com/auth
    ngrok.set_auth_token("ENTER YOUR AUTH TOKEN")

    # Create ngrok tunnel to port 5000
    public_url = ngrok.connect(5000).public_url
    print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:5000'")
    return public_url

# Run the app
if __name__ == '__main__':
    public_url = setup_ngrok()
    app.run(port=5000)




 * ngrok tunnel 'https://9a07-34-16-15-111.ngrok-free.app' -> 'http://127.0.0.1:5000'
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'type': 1, 'start_date': '01/08/2024', 'end_date': '31/08/2024', 'response': 'Here is the plot for employee presence for the month of August 2024.'}
inside type 1


INFO:werkzeug:127.0.0.1 - - [20/Jan/2025 11:45:53] "POST /process HTTP/1.1" 200 -


Here is the plot for employee presence for the month of August 2024.
{'type': 1, 'start_date': '01/08/2025', 'end_date': '07/08/2025', 'response': 'Here is the plot for employee absence for the first week of August.'}
inside type 1


INFO:werkzeug:127.0.0.1 - - [20/Jan/2025 11:46:47] "POST /process HTTP/1.1" 200 -


Here is the plot for employee absence for the first week of August.


INFO:werkzeug:127.0.0.1 - - [20/Jan/2025 11:57:33] "POST /process HTTP/1.1" 200 -


{'type': 2, 'start_date': '20/01/2025', 'end_date': '20/04/2025', 'response': 'Based on your input, the project will be completed on {estimated_date}.'}
inside type 2
Based on your input, the project will be completed on {estimated_date}.
2025-06-06 00:00:00
